# Collect all cx translations

In [45]:
import pandas as pd
import numpy as np
import math

import datetime as dt

from wmfdata import hive, mariadb, spark

In [2]:
#collect content translation and associated MT engine
#need to query cx_corpus table and cross it with cx_translations table and then group and sort

query = """ 
    SELECT
    translation_start_timestamp AS translation_start_time,
    translation_id,
    translation_target_revision_id  AS target_revision_id,
    translation_target_title AS page_title,
    json_extract(translation_progress, '$.mt') AS mt_translated_percent, 
    json_extract(translation_progress, '$.human') AS human_translated_percent, 
    translation_status AS translation_status,
    CASE
    WHEN cc.cxc_origin = 'source' THEN 'source'
    WHEN cc.cxc_origin = 'user' THEN 'final'
    ELSE 'mt_engine'
    END AS translation_type,
    cc.cxc_origin AS mt_service,
    translation_source_language AS source_language,
    translation_target_language AS target_language
    FROM 
    cx_translations
    JOIN 
    cx_corpora AS cc
    ON translation_id = cc.cxc_translation_id
"""


In [3]:
mt_data = mariadb.run(commands = query, dbs = "wikishared", use_x1 = False, format="pandas", date_col=None,
 index_col=None)

In [4]:
#save data to csv
mt_data.to_csv('mt_data.csv', index = False)